In [7]:
from metaflow import Flow
flow = Flow('main_flow')
for run in flow:
    print(run)

Run('main_flow/3')
Run('main_flow/2')
Run('main_flow/1')


In [21]:
from metaflow import Step
latest_run = Flow('main_flow').latest_run
step = latest_run['deploy']
y_pred = step.task.data.y_pred

print(y_pred)
print(type(y_pred))

boxes = step.task.data.boxes
print(boxes)

box_list = step.task.data.box_list
print(box_list)

classes = step.task.data.classes
print(classes)

{'boxes': <tf.RaggedTensor []>, 'classes': <tf.Tensor: shape=(0,), dtype=float32, numpy=array([], dtype=float32)>}
<class 'dict'>
<tf.RaggedTensor []>
[]
tf.Tensor([], shape=(0,), dtype=float32)
